In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report 

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv('/kaggle/input/heart-disease-uci/heart.csv')
X, y = data.drop(['target'], axis=1), data['target']
from sklearn.preprocessing import StandardScaler

def scale_a_column(data, label_column='target'):
    scaler = StandardScaler()
    df1 = data.drop([label_column], axis=1)
    res = scaler.fit_transform(df1)
    df1 = pd.DataFrame(res, columns=df1.columns)
    df1[label_column] = data[label_column]
    return df1
scaled_data = scale_a_column(data)
X_scaled, y_scaled = scaled_data.drop(['target'], axis=1), scaled_data['target']

In [ ]:
data

In [ ]:
import plotly.graph_objects as go

def plot_3d(data, f1, f2, f3, target='target'):
    PLOT = go.Figure()
    for C in list(data[target].unique()):    
        PLOT.add_trace(go.Scatter3d(x = data[data[target] == C][f1],
                                    y = data[data[target] == C][f2],
                                    z = data[data[target] == C][f3],
                                    mode = 'markers', marker_size = 8, marker_line_width = 1,
                                    name = f'{target} ' + str(C)))

    PLOT.update_layout(width = 800, height = 800, autosize = True, showlegend = True,
                       scene = dict(xaxis=dict(title = f1, titlefont_color = 'black'),
                                    yaxis=dict(title = f2, titlefont_color = 'black'),
                                    zaxis=dict(title = f3, titlefont_color = 'black')),
                       font = dict(family = "Gilroy", color  = 'black', size = 12))
    PLOT.show()


In [ ]:
def plot_2d(data, x, y, figsize=(9, 5), hue='target'):
    plt.figure(figsize = figsize)
    sns.swarmplot(x = x, y = y, hue=hue,data=data)
    plt.legend()
    plt.title(f"{x} -- {y} -- {hue}")
    plt.show()

In [ ]:
#for i in range(X_scaled.shape[1]):
#    for j in range(i+1, X_scaled.shape[1]):
#            plot_2d(scaled_data, scaled_data.columns[i], scaled_data.columns[j])

In [ ]:
def run_kmean(X_scaled, y, n_clusters=4, compare_cluster=False):
    kmeans = KMeans(
        init="random",
        n_clusters=n_clusters,
        n_init=10,
        max_iter=300,
        random_state=42)

    X_kmeans = X_scaled.copy()
    kmeans.fit(X_kmeans)
    
    X_kmeans = X_scaled.copy()
    X_kmeans['cluster'] = kmeans.labels_
    
    if compare_cluster:
        d = pd.concat([X_kmeans, y], axis=1)
        corr1 = d['cluster'].corr(d['target'])
        d['cluster'] = 1 - d['cluster']
        corr2 = d['cluster'].corr(d['target'])
        print(corr1, corr2)
    else:
        print(pd.concat([X_kmeans, y], axis=1))
    #print(X_kmeans)
    return X_kmeans

In [ ]:
def run_em(X_scaled, y, n_components=4, compare_cluster=False):
    X_GM = X_scaled.copy()
    gm = GaussianMixture(n_components=n_components, random_state=0).fit(X_GM)
    labels = gm.predict(X_GM)

    X_GM = X_scaled.copy()
    X_GM['cluster'] = labels
    if compare_cluster:
        d = pd.concat([X_GM, y], axis=1)
        print(d['cluster'].corr(d['target']))
    else:
        print(pd.concat([X_GM, y], axis=1))
    #print(X_GM)    
    return X_GM

In [ ]:
from sklearn import decomposition

def run_pca(X, y):
    pca = decomposition.PCA(n_components=int(X_scaled.shape[1]*0.7))
    pca = decomposition.PCA(n_components=3)
    pca.fit(X)
    X_pca = pca.transform(X)

    X_pca=pd.DataFrame(X_pca)
    X_pca.index = X.index
    X_pca.columns = [f'col{i+1}' for i in range(X_pca.shape[1])]
    data_pca_for_plot = pd.concat([X_pca, y], axis=1)

    return X_pca

In [ ]:
from sklearn import decomposition

def run_ica(X, y):
    ica = decomposition.FastICA(n_components=int(X_scaled.shape[1]*0.7))
    ica.fit(X)
    X_ica = ica.transform(X)

    X_ica = pd.DataFrame(X_ica)
    X_ica.index = X.index
    X_ica.columns = [f'col{i+1}' for i in range(X_ica.shape[1])]
    data_ica_for_plot=pd.concat([X_ica, y], axis=1)

    return X_ica

In [ ]:
from sklearn import random_projection

def run_random_project(X_scaled, y):
    transformer = random_projection.GaussianRandomProjection(n_components=int(X_scaled.shape[1]*0.7))
    #transformer = random_projection.GaussianRandomProjection()
    X_rp = transformer.fit_transform(X_scaled)
    X_rp = pd.DataFrame(X_rp)
    X_rp.columns = [f'col{i+1}' for i in range(X_rp.shape[1])]

    data_rp_for_plot=pd.concat([X_rp, y], axis=1)

    for i in range(1, data_rp_for_plot.shape[1]):
        for j in range(i+1, data_rp_for_plot.shape[1]):
            for k in range(j+1, data_rp_for_plot.shape[1]):
                break
                plot_3d(data_rp_for_plot, f'col{i}', f'col{j}', f'col{k}', 'Drug')
    return X_rp

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

def run_tree_based_feature_selection(X, y):
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(X, y)
    #pd.DataFrame.from_dict(data, orient='index')
    wieghts_df = pd.DataFrame.from_dict(dict(zip(X.columns, list(clf.feature_importances_))), orient='index', columns=['Weight'])
    threshhold = min(clf.feature_importances_) * 1.01
    threshhold = None
    model = SelectFromModel(clf, max_features=4, threshold=threshhold, prefit=True)
    X_selected = model.transform(X)

    X_selected = pd.DataFrame(X_selected)
    X_selected.index = X.index
    X_selected.columns = [f'col{i+1}' for i in range(X_selected.shape[1])]
    print(wieghts_df.transpose())
    
    data_rp_for_plot=pd.concat([X_selected, y], axis=1)
    print(data_rp_for_plot)

    for i in range(1, data_rp_for_plot.shape[1]):
        for j in range(i+1, data_rp_for_plot.shape[1]):
            for k in range(j+1, data_rp_for_plot.shape[1]):
                break
                plot_3d(data_rp_for_plot, f'col{i}', f'col{j}', f'col{k}', 'target')
    
    return X_selected

In [ ]:
X_scaled_em = run_em(X_scaled, y)
X_scaled_kmean = run_kmean(X_scaled, y)

In [ ]:
x_ica = run_ica(X_scaled, y)
x_pca = run_pca(X_scaled, y)
x_rp  = run_random_project(X_scaled, y)
x_tbs = run_tree_based_feature_selection(X_scaled, y)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
n_max = 50
n_start = 2
scores = []
inertia_list = np.empty(n_max)

for i in range(n_start,n_max):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(x_pca)
    inertia_list[i] = kmeans.inertia_
    scores.append(silhouette_score(x_pca, kmeans.labels_))

n_max_shift = 2  # find maximum after this index of score
n_clusters = np.argmax(scores[n_max_shift:])+(n_start+n_max_shift) # it's my upgrade
plt.plot(range(0,n_max),inertia_list,'-o')
plt.xlabel('Number of cluster')
plt.axvline(x=n_clusters, color='blue', linestyle='--')
plt.ylabel('Inertia')
plt.show()

plt.plot(range(2,n_max), scores);
plt.title('Results KMeans')
plt.xlabel('n_clusters');
plt.axvline(x=n_clusters, color='blue', linestyle='--')
plt.ylabel('Silhouette Score');
plt.show()

In [ ]:
n_clusters

In [ ]:
print(x_tbs)

In [ ]:
x_ica_with_kmean = run_kmean(x_ica, y)
x_pca_with_kmean = run_kmean(x_pca, y)
x_rp_with_kmean = run_kmean(x_rp, y)
x_tbs_with_kmean = run_kmean(x_tbs, y)

In [ ]:
x_ica_with_em = run_em(x_ica, y)
x_pca_with_em = run_em(x_pca, y)
x_rp_with_em = run_em(x_rp, y)
x_tbs_with_em = run_em(x_tbs, y)

In [ ]:
def run_test(X, y ,clf_name, hidden_layers = 100):
    clf = MLPClassifier(random_state=1, max_iter=2000, hidden_layer_sizes=hidden_layers)
    X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=101)
    clf=clf.fit(X_train, y_train)
    return (clf_name, clf.score(X_test, y_test))

In [ ]:
res = [ run_test(X, y, 'Basic'), 
run_test(x_ica, y, 'ICA'),
run_test(x_pca, y, 'PCA'), 
run_test(x_rp, y, 'Randomized Projection'),
run_test(x_tbs, y, 'Tree based selection')]

pd.DataFrame.from_dict({item[0] : [item[1]] for item in res})

In [ ]:
res = [ run_test(X, y, 'Basic'), 
run_test(x_ica_with_kmean, y, 'ICA'),
run_test(x_pca_with_kmean, y, 'PCA'), 
run_test(x_rp_with_kmean, y, 'Randomized Projection'),
run_test(x_tbs_with_kmean, y, 'Tree based selection')]

pd.DataFrame.from_dict({item[0] : [item[1]] for item in res})

In [ ]:
res = [ run_test(X, y, 'Basic'), 
run_test(x_ica_with_em, y, 'ICA'),
run_test(x_pca_with_em, y, 'PCA'), 
run_test(x_rp_with_em, y, 'Randomized Projection'),
run_test(x_tbs_with_em, y, 'Tree based selection')]

pd.DataFrame.from_dict({item[0] : [item[1]] for item in res})

In [ ]:
sns.set_style('darkgrid')
axes = pd.plotting.scatter_matrix(data, alpha = 0.3, figsize = (10,7), diagonal = 'kde' ,s=80)
corr = data.corr().values

plt.xticks(fontsize =10,rotation =0)
plt.yticks(fontsize =10)
for ax in axes.ravel():
    ax.set_xlabel(ax.get_xlabel(),fontsize = 15, rotation = 60)
    ax.set_ylabel(ax.get_ylabel(),fontsize = 15, rotation = 60)
# put the correlation between each pair of variables on each graph
for i, j in zip(*np.triu_indices_from(axes, k=1)):
    axes[i, j].annotate("%.3f" %corr[i, j], (0.8, 0.8), xycoords="axes fraction", ha="center", va="center")

In [ ]:
x_ica_with_kmean = run_kmean(x_ica, y, 2, True)
x_pca_with_kmean = run_kmean(x_pca, y, 2, True)
x_rp_with_kmean = run_kmean(x_rp, y, 2, True)
x_tbs_with_kmean = run_kmean(x_tbs, y, 2, True)

In [ ]:
x_ica_with_em = run_em(x_ica, y, 2, True)
x_pca_with_em = run_em(x_pca, y, 2, True)
x_rp_with_em = run_em(x_rp, y, 2, True)
x_tbs_with_em = run_em(x_tbs, y, 2, True)